# Titanic Survival Model 
## Predicting the chances of survival for passengers on the Titanic using Classification

The aim of this project is to analyze the data given about passengers on the Titanic, and using the data predict the chances of survival for a particular passenger.
2 datasets are given : train.csv and test.csv
we can train our model using train.csv which has both the passenger details and if they survived or not, and then apply our model on test.csv which only has the passenger details and no information about if the passenger survived or not. 

## Loading important libraries that we may use: 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Loading out training data into a Data Frame : 